In [179]:
import ast
import pandas as pd
import numpy as np
import torch
from tensorflow import keras
from tdc.single_pred.adme import ADME
from tdc import Evaluator
from tqdm.notebook import tqdm, trange
from torch.utils.data import TensorDataset, DataLoader
from rdkit import Chem
from rdkit.Chem import AllChem
from matplotlib import pyplot as plt
from IPython import display
from rdkit.Chem import BondType
from rdkit.Chem.Draw import MolsToGridImage
from torch import tensor

In [180]:
csv_path = keras.utils.get_file(
    "250k_rndm_zinc_drugs_clean_3.csv",
    "https://raw.githubusercontent.com/aspuru-guzik-group/chemical_vae/master/models/zinc_properties/250k_rndm_zinc_drugs_clean_3.csv",
)

df = pd.read_csv("~/.keras/datasets/250k_rndm_zinc_drugs_clean_3.csv")
df["smiles"] = df["smiles"].apply(lambda s: s.replace("\n", ""))
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249455 entries, 0 to 249454
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   smiles  249455 non-null  object 
 1   logP    249455 non-null  float64
 2   qed     249455 non-null  float64
 3   SAS     249455 non-null  float64
dtypes: float64(3), object(1)
memory usage: 7.6+ MB


In [181]:
SMILE_CHARSET = '[None, "C", "B", "F", "I", "H", "O", "N", "S", "P", "Cl", "Br"]'

bond_mapping = {"SELF": 0, "SINGLE": 1, "DOUBLE": 2, "TRIPLE": 3, "AROMATIC": 4}
bond_mapping.update(
    {0: None, 1: BondType.SINGLE, 2: BondType.DOUBLE, 3: BondType.TRIPLE, 4: BondType.AROMATIC}
)
SMILE_CHARSET = ast.literal_eval(SMILE_CHARSET)
MAX_BONDNUM = 150
MAX_MOLSIZE = 64 #max(df["smiles"].str.len())
SMILE_to_index = dict((c, i) for i, c in enumerate(SMILE_CHARSET))
index_to_SMILE = dict((i, c) for i, c in enumerate(SMILE_CHARSET))
atom_mapping = dict(SMILE_to_index)
atom_mapping.update(index_to_SMILE)

BATCH_SIZE = 1
EPOCHS = 500

VAE_LR = 5e-4
NUM_ATOMS = 32  # Maximum number of atoms

ATOM_DIM = len(SMILE_CHARSET)  # Number of atom types
BOND_DIM = 5   # Number of bond types
LATENT_DIM = 128  # Size of the latent space

In [182]:
import sklearn
from sklearn import model_selection
from torch.utils.data import DataLoader

train_data, test_data = model_selection.train_test_split(df, test_size=0.2)
train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)
test_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=False)

In [183]:
def molecule_to_graph(molecule):
    # Converts SMILES to molecule object
    
    mol_atoms = molecule.GetNumAtoms()    # Initialize adjacency and feature tensor
    num_of_bonds = molecule.GetNumBonds()
    edge_index = [[],[]]
    edge_features = []
    features = np.zeros((NUM_ATOMS, ATOM_DIM))

    # loop over each atom in molecule
    for atom in molecule.GetAtoms():
        atom_idx = atom.GetIdx()
        atom_type = atom_mapping[atom.GetSymbol()]
        atomic_num = atom.GetAtomicNum()
        degree = atom.GetDegree()
        #mass = atom.GetMass()
        edge_index[0].append(atom_idx)
        edge_index[1].append(atom_idx)
        edge_embbeding = list(np.eye(BOND_DIM)[0])
        edge_features.append(edge_embbeding)
        #chem_features = [atomic_num, degree] #, mass]
        #print(atom_idx)
        features[atom_idx] = np.eye(ATOM_DIM)[atom_type] #np.concatenate((np.eye(ATOM_DIM)[atom_type][:-2], chem_features), axis=0)
    for i in range(mol_atoms, NUM_ATOMS):
        features[i][0] = 1
        edge_index[0].append(i)
        edge_index[1].append(i)
        edge_embbeding = list(np.eye(BOND_DIM)[0])
        edge_features.append(edge_embbeding)
        # loop over one-hop neighbors
    for bond in molecule.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        bond_type_idx = bond_mapping[bond.GetBondType().name]
        edge_index[0] = list(np.append(edge_index[0], [i, j]))
        edge_index[1] = list(np.append(edge_index[1], [j, i]))
        edge_embbeding = list(np.eye(BOND_DIM)[bond_type_idx])
        edge_features.append(edge_embbeding)
        edge_features.append(edge_embbeding)

    
    return tensor(features), tensor(edge_index), tensor(edge_features)


def graph_to_molecule(graph):
    # Unpack graph
    adjacency, features = graph

    # RWMol is a molecule object intended to be edited
    molecule = Chem.RWMol()

    # Remove "no atoms" & atoms with no bonds
    keep_idx = np.where(
        (np.argmax(features, axis=1) != ATOM_DIM - 1)
        & (np.sum(adjacency[:-1], axis=(0, 1)) != 0)
    )[0]
    features = features[keep_idx]
    adjacency = adjacency[:, keep_idx, :][:, :, keep_idx]

    # Add atoms to molecule
    for atom_type_idx in np.argmax(features, axis=1):
        atom = Chem.Atom(atom_mapping[atom_type_idx])
        _ = molecule.AddAtom(atom)

    # Add bonds between atoms in molecule; based on the upper triangles
    # of the [symmetric] adjacency tensor
    (bonds_ij, atoms_i, atoms_j) = np.where(np.triu(adjacency) == 1)
    for (bond_ij, atom_i, atom_j) in zip(bonds_ij, atoms_i, atoms_j):
        if atom_i == atom_j or bond_ij == BOND_DIM - 1:
            continue
        bond_type = bond_mapping[bond_ij]
        molecule.AddBond(int(atom_i), int(atom_j), bond_type)

    # Sanitize the molecule; for more information on sanitization, see
    # https://www.rdkit.org/docs/RDKit_Book.html#molecular-sanitization
    flag = Chem.SanitizeMol(molecule, catchErrors=True)
    # Let's be strict. If sanitization fails, return None
    if flag != Chem.SanitizeFlags.SANITIZE_NONE:
        return None

    return molecule

def adj_to_index(adj_matrix):
    edge_index=[[], []]
    for i, row in enumerate(adj_matrix):
        adj_matrix[i][i]=1 
        for j in range(len(row)):
            if row[j] == 1:
                edge_index[0].append(i)
                #edge_index[0].append(j)
                #edge_index[1].append(i)
                edge_index[1].append(j)
    return torch.tensor(edge_index)

def edge_idx_to_adj(edge_index):
    adj = np.zeros((NUM_ATOMS, NUM_ATOMS))
    for i in range(edge_index.shape[1]):
        k = edge_index[0][i]
        j = edge_index[1][i]
        adj[k][j] = 1
    return torch.tensor(adj)

In [184]:
from sklearn.discriminant_analysis import StandardScaler
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader


class MyDataLoader():
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __call__(self, df):
        graphs = []
        labels = []
        for i, row in df.iterrows():
            smiles = row["smiles"]
            molecule = Chem.MolFromSmiles(smiles)
            if molecule.GetNumAtoms() < 32:
                features, edge_index, edge_attr = molecule_to_graph(molecule)
            #print(features, edge_index, edge_attr)  
                graphs.append((features, edge_index, edge_attr))
                labels.append(molecule)
            
        return [Data(
            features=features.float(), 
            edge_index=edge_index.int(), 
            edge_attr=edge_attr.float(),
            mol=labels)
            for ((features, edge_index, edge_attr), labels) in zip(graphs, labels)]

In [185]:
data = MyDataLoader()
data = data(df[:16])


In [186]:
data[0].features

tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1

In [187]:
from torch_geometric.loader import DataLoader
loader = DataLoader(data, batch_size=BATCH_SIZE)

In [188]:
for i in loader:
    print(i.batch)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0,

In [189]:
from torch_geometric.nn import GATConv
from torch_geometric.nn.pool import global_mean_pool
import torch.nn.functional as F
from torch.nn.functional import dropout
import torch

HIDDEN_SIZE=512

class Encoder(torch.nn.Module):
    def __init__(self, hidden_size=HIDDEN_SIZE, atom_dim = ATOM_DIM, latent_dim=LATENT_DIM):
        super(Encoder, self).__init__()
        torch.manual_seed(12345)

        self.conv = [
            GATConv(atom_dim, hidden_size),
            GATConv(hidden_size, hidden_size),
            GATConv(hidden_size, hidden_size),
            GATConv(hidden_size, hidden_size),
            GATConv(hidden_size, hidden_size),
            GATConv(hidden_size, hidden_size)
        ]

        self.dense = [
            torch.nn.Linear(in_features=hidden_size, out_features=hidden_size)
        ]
        self.mean = torch.nn.Linear(in_features=hidden_size, out_features=latent_dim)
        self.std = torch.nn.Linear(in_features=hidden_size, out_features=latent_dim)



    def forward(self, x, edge_index, edge_attr, batch): 
        for conv_layer in self.conv:
            x = conv_layer(x, edge_index, edge_attr)
            x = torch.nn.ELU()(x)

        x = global_mean_pool(x, batch=batch, size = x.shape[0])[0]
        #print(nodes.shape, nodes)
        x = torch.flatten(x)
        for layer in self.dense:
            x = layer(x, edge_index, edge_attr)
            x = torch.nn.ELU()(x)

        mean, std = self.mean(x, edge_index, edge_attr), self.std(x, edge_index, edge_attr)

        return mean, std
    
    def loss():
        pass




class Decoder(torch.nn.Module):
    def __init__(self, hidden_size=HIDDEN_SIZE, atom_dim = ATOM_DIM, latent_dim=LATENT_DIM) -> None:
        super(Decoder, self).__init__()
        torch.manual_seed(12345)
        self.edges_transform = torch.nn.Linear(in_features=latent_dim, out_features=int(NUM_ATOMS)*int(NUM_ATOMS))

        self.nodes_linear1 = torch.nn.Linear(in_features=latent_dim, out_features=NUM_ATOMS)
        self.nodes_linear2 = torch.nn.Linear(in_features=NUM_ATOMS, out_features=int(NUM_ATOMS)*int(ATOM_DIM))

    def forward(self, nodes_encoded, edges_encoded, batch):
        edges = self.edges_transform(edges_encoded)
        #edges = torch.sigmoid(edges)

        edges = torch.reshape(edges, (NUM_ATOMS, NUM_ATOMS)) + torch.transpose_copy(torch.reshape(edges, (NUM_ATOMS, NUM_ATOMS)), 0, 1)
        #adjacency = torch.matmul(torch.unsqueeze(edges, 1), torch.unsqueeze(edges, 0))
        #edges = torch.softmax(edges, 0, float)

        nodes = self.nodes_linear1(nodes_encoded)
        nodes = self.nodes_linear2(nodes)
        nodes = torch.reshape(nodes, (NUM_ATOMS, ATOM_DIM))
        #nodes = F.one_hot(torch.argmax(nodes, dim=1), ATOM_DIM)
        return torch.tensor(nodes, dtype=float), edges

    def loss():
        pass

 

class GraphNeuralNetwork(torch.nn.Module):  # TODO: assign hyperparameters to attributes and define the forward pass
    def __init__(self, encoder, decoder, hidden_size, atom_dim = ATOM_DIM, latent_dim=LATENT_DIM):
        super(GraphNeuralNetwork, self).__init__()
        torch.manual_seed(12345)

        self.encoder = Encoder(hidden_size)
        self.decoder = Decoder(hidden_size)

        #decode edges:
        
    
    def encode(self, features, edge_index, edge_attr, batch):
        return self.encoder(features, edge_index, edge_attr, batch)

    def decode(self, nodes_encoded, edges_encoded, batch):
        return self.decoder(nodes_encoded, edges_encoded, batch)

    def forward(self, features, adj, edge_attr):
        nodes_enc, edges_enc = self.encode(features, adj, edge_attr)
        nodes_dec, edges_dec = self.decode(nodes_enc, edges_enc)

        return nodes_dec, edges_dec
    

    def compute_loss(self, nodes_generated, nodes_real, edges_generated, edges_real):

        loss = torch.nn.CrossEntropyLoss()
        nodes_loss = loss(nodes_generated, nodes_real)
        edges_loss = loss(edges_generated, edge_idx_to_adj(edges_real))
        return nodes_loss + edges_loss





In [190]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [191]:
encoder = Encoder(hidden_size=HIDDEN_SIZE)
decoder = Encoder(hidden_size=HIDDEN_SIZE)

encoder=encoder.float()
decoder = decoder.float()
g = GraphNeuralNetwork(hidden_size=512, encoder=encoder, decoder=decoder)
#g=g.float()


In [197]:
from numpy import int64


def train(model, train_loader, valid_loader, l_r = 0.00001, epochs = 100):
        # hyperparameters definition
        hidden_size = 512
        learning_rate = l_r
    
        #model = GraphNeuralNetwork(hidden_size, -1, 1)  # TODO: you can add more hyperparameters if needed
        #model.train()
    
        optimizer = torch.optim.Adam(params=model.parameters(), lr= learning_rate)
        for epoch in trange(1, epochs + 1, leave=False):
            model.train(True)
            for data in train_loader:
                real_features, real_edge_index, real_edge_attr, batch, mol = data.features, data.edge_index.to(dtype=torch.int64), data.edge_attr, data.batch, data.mol
                #print(real_edge_index.shape)
                x_encoded = model.encode(real_features, real_edge_index, real_edge_attr, batch)
                #real_features = F.one_hot(torch.argmax(real_features, dim=1), ATOM_DIM)
                x_decoded = model.decode(x_encoded[0], x_encoded[1], batch)
                #print(x_decoded[0].shape, x_decoded[1].shape)
                #print(x_decoded[0].shape, real_features.shape)
                loss = model.compute_loss(x_decoded[0], real_features, x_decoded[1], real_edge_index)
                #print(x_decoded[0], real_features, x_decoded[1], real_edge_index)
                print(loss)
                loss.backward()
                optimizer.step() 
                optimizer.zero_grad()
                model.zero_grad()

            # for data in tqdm(valid_loader):
            #     x, edge_index, batch, y = data.x, data.edge_index, data.batch, data.y
            #     preds = model(x, edge_index, batch)
            #     preds_batches.append(preds.detach().numpy())
            #     y_valid_batches.append(y.detach().numpy())
            
        #return self

In [199]:

train(g, train_loader=loader, valid_loader=test_data_loader)

  0%|          | 0/100 [00:00<?, ?it/s]

tensor(6.7168, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(6.4438, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(7.4346, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(7.7023, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(7.2852, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(5.7102, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(5.9390, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(6.2851, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(7.1043, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(5.7992, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(7.1345, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(6.8752, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(7.2045, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(7.0444, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(7.5179, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(6.7015, dtype=torch.float64, grad_fn=<AddBackward0>)
tensor(6.4336, dtype=torch.float64, grad

In [194]:
for i in loader:
    print(i)

DataBatch(edge_index=[2, 84], edge_attr=[84, 5], features=[32, 12], mol=[1], batch=[32], ptr=[2])
DataBatch(edge_index=[2, 78], edge_attr=[78, 5], features=[32, 12], mol=[1], batch=[32], ptr=[2])
DataBatch(edge_index=[2, 96], edge_attr=[96, 5], features=[32, 12], mol=[1], batch=[32], ptr=[2])
DataBatch(edge_index=[2, 98], edge_attr=[98, 5], features=[32, 12], mol=[1], batch=[32], ptr=[2])
DataBatch(edge_index=[2, 92], edge_attr=[92, 5], features=[32, 12], mol=[1], batch=[32], ptr=[2])
DataBatch(edge_index=[2, 66], edge_attr=[66, 5], features=[32, 12], mol=[1], batch=[32], ptr=[2])
DataBatch(edge_index=[2, 72], edge_attr=[72, 5], features=[32, 12], mol=[1], batch=[32], ptr=[2])
DataBatch(edge_index=[2, 80], edge_attr=[80, 5], features=[32, 12], mol=[1], batch=[32], ptr=[2])
DataBatch(edge_index=[2, 90], edge_attr=[90, 5], features=[32, 12], mol=[1], batch=[32], ptr=[2])
DataBatch(edge_index=[2, 68], edge_attr=[68, 5], features=[32, 12], mol=[1], batch=[32], ptr=[2])
DataBatch(edge_index

In [195]:

print(torch.cuda.is_available())

False
